# Introduction

Purpose: Develop a model that can detect small objects, such as birds, both stationary and in motion from a Rpi 5. 

Use Case: I have a RPi with a camera that is monitoring objects outside of my apartment unit. I would like to specifically detect small animals such as birds, squirrels and foxes. I do not care about detecting other classes such as people or people with dogs. 

Architecture:

- Apply transfer learning to a YOLO11 NCNN model that is optimized for RPi deployment
- Trade Study: SAHI to train model on different splits of the image
- Apply data augmentation
- Tiling during pre-processing and inference
- Nice to have: Omit classes


Data Acquisition
- figure out where to get a relevant data set???
      - create my own?

Data Annotation
- already done by data set...?

Data Preprocessing
- resize if necessary
- normalize if necessary
- split
- cleaning
- tiling?

Data Augmentation
- Augment data in pipeline

Model Selection
- YOLO NCNN (either v11 or maybe an earlier one?)
- use pre-trained weights?

Training
- select appropriate loss function
- use adam optimizer
- Hyperparameters and tuning

Validation 
- learning rate schedulers and early stopping
- hyperparameter tuning
- asses validation loss accuracy and loss

Evaluation
- use test data to assess generalization....

Deploy
- ???

In [1]:
pip install ultralytics ncnn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 86.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
  

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from ultralytics import YOLO

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


# Data Augmentation and Training

Decided not to do tiling since we'll be using SAHI at inference. 

In [3]:
model = YOLO("yolo11n.pt", task='detect')
data_path = '/kaggle/input/bird-train-valid-test-split/data.yaml'
model.model

100%|██████████| 5.35M/5.35M [00:00<00:00, 75.0MB/s]


DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C3k2(
      (cv1): Conv(
        (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(48, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
   

In [4]:
def train_yolo11_advanced(data_path, pretrained_model):
    """
    Advanced YOLO11 training with Adam optimizer, hyperparameter tuning,
    learning rate scheduling, and early stopping
    """

    model = pretrained_model
    
    # Training configuration
    train_config = {
        # Dataset
        'data': data_path,
        
        # Training parameters
        'epochs': 300,
        'batch': 16,
        'imgsz': 640,
        'device': 0,  # GPU device, or 'cpu'
        
        # Optimizer settings
        'optimizer': 'Adam',  # Adam optimizer
        'lr0': 0.001,         # Initial learning rate
        'lrf': 0.01,          # Final learning rate (lr0 * lrf)
        'momentum': 0.937,    # Momentum (for SGD, but affects Adam's beta1)
        'weight_decay': 0.0005,  # Weight decay (L2 regularization)
        
        # Learning rate scheduler
        'cos_lr': True,       # Use cosine learning rate scheduler
        'warmup_epochs': 3,   # Warmup epochs
        'warmup_momentum': 0.8,  # Warmup momentum
        'warmup_bias_lr': 0.1,   # Warmup bias learning rate
        
        # Early stopping
        'patience': 50,       # Epochs to wait for improvement
        'save_period': 10,    # Save checkpoint every N epochs
        
        # Data augmentation
        'hsv_h': 0.015,       # HSV-Hue augmentation
        'hsv_s': 0.5,         # HSV-Saturation augmentation
        'hsv_v': 0.4,         # HSV-Value augmentation
        'degrees': 10.0,      # Rotation degrees
        'translate': 0.1,     # Translation fraction
        'scale': 0.2,         # Scaling factor
        'shear': 1.0,         # Shear degrees
        'perspective': 0.0,   # Perspective transformation
        'flipud': 0.0,        # Vertical flip probability
        'fliplr': 0.5,        # Horizontal flip probability
        'mosaic': 0.75,        # Mosaic augmentation probability
        'mixup': 0.1,         # MixUp augmentation probability
        'copy_paste': 0.1,    # Copy-paste augmentation probability
        
        # Loss function weights (YOLO11 uses composite loss)
        'box': 7.5,           # Box loss weight
        'cls': 0.5,           # Classification loss weight
        'dfl': 1.5,           # Distribution focal loss weight
        
        # Training optimization
        'amp': True,          # Automatic Mixed Precision
        'fraction': 1.0,      # Fraction of dataset to train on
        'profile': False,     # Profile training speed
        'freeze': None,      # Freeze layers (list of layer indices or None)
        
        # Validation
        'val': True,          # Validate during training
        'split': 'val',       # Dataset split to use for validation
        'save_json': False,   # Save results to JSON
        'save_hybrid': False, # Save hybrid version of labels
        'conf': 0.25,         # Confidence threshold for predictions
        'iou': 0.7,           # IoU threshold for NMS
        'max_det': 300,       # Maximum detections per image
        'half': False,        # Use half precision
        'dnn': False,         # Use OpenCV DNN backend
        'plots': True,        # Generate plots during training
        
        # Logging
        'verbose': True,      # Verbose output
        'seed': 0,            # Random seed for reproducibility
        'deterministic': True, # Deterministic training
        'single_cls': False,  # Single class training
        'rect': False,        # Rectangular training
        'resume': False,      # Resume training from last checkpoint
        'cache': None,        # Cache dataset ('ram', 'disk', or None)
        'multi_scale': False, # Multi-scale training
        'overlap_mask': True, # Overlap masks during training
        'mask_ratio': 4,      # Mask downsample ratio
        'dropout': 0.0,       # Dropout rate
    }

    # Train the model
    results = model.train(**train_config)
    
    return results

results = train_yolo11_advanced(data_path, model)


WARNING ⚠️ 'save_hybrid' is deprecated and will be removed in in the future.
Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=None, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=0.25, copy_paste=0.1, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/kaggle/input/bird-train-valid-test-split/data.yaml, degrees=10.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=300, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.5, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=0.75, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, 

100%|██████████| 755k/755k [00:00<00:00, 17.9MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /kaggle/input/bird-train-valid-test-split/train/labels... 6755 images, 1036 backgrounds, 0 corrupt: 100%|██████████| 6755/6755 [00:16<00:00, 418.96it/s]


WARNING ⚠️ train: Cache directory /kaggle/input/bird-train-valid-test-split/train is not writeable, cache not saved.
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 9.2±3.8 MB/s, size: 40.0 KB)


...

    300/300      4.15G      1.796     0.8594     0.7753          2        640: 100%|██████████| 423/423 [01:11<00:00,  5.90it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 61/61 [00:10<00:00,  5.73it/s]

                   all       1924       5696       0.88      0.297      0.596      0.305



300 epochs completed in 8.042 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 5.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 5.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 61/61 [00:11<00:00,  5.20it/s]
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all       1924       5696       0.88      0.296      0.595      0.305
Speed: 0.2ms preprocess, 2.4ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/train


# Validate, Test and Export for RPI5

In [5]:
# Export Model
model.export(format="ncnn")  # creates 'yolo11n_ncnn_model'

# Validate the model
metrics = model.val(data="/kaggle/input/bird-train-valid-test-split/data.yaml", split="val")  # no arguments needed, dataset and settings remembered
metrics.box.map  
metrics.box.map50  
metrics.box.map75  
metrics.box.maps  

# Test Model
metrics = model.val(data="/kaggle/input/bird-train-valid-test-split/data.yaml", split="test")  # no arguments needed, dataset and settings remembered
metrics.box.map  
metrics.box.map50  
metrics.box.map75  
metrics.box.maps



Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.00GHz)
💡 ProTip: Export to OpenVINO format for best performance on Intel hardware. Learn more at https://docs.ultralytics.com/integrations/openvino/
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs

PyTorch: starting from 'runs/detect/train/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (5.2 MB)

TorchScript: starting export with torch 2.6.0+cu124...
TorchScript: export success ✅ 2.4s, saved as 'runs/detect/train/weights/best.torchscript' (10.4 MB)

NCNN: starting export with NCNN 1.0.20250503...
WARNING ⚠️ NCNN: PNNX not found. Attempting to download binary file from https://github.com/pnnx/pnnx/.
Note PNNX Binary file must be placed in current working directory or in /usr/local/lib/python3.11/dist-packages/ultralytics. See PNNX repo for full installation instructions.
NCNN: successfully found latest PNNX asset file pnnx-20250530-linux.zi

100%|██████████| 23.6M/23.6M [00:00<00:00, 128MB/s]
Unzipping pnnx-20250530-linux.zip to /kaggle/working/pnnx-20250530-linux...: 100%|██████████| 3/3 [00:00<00:00,  6.42file/s]

NCNN: running '/usr/local/lib/python3.11/dist-packages/ultralytics/pnnx runs/detect/train/weights/best.torchscript ncnnparam=runs/detect/train/weights/best_ncnn_model/model.ncnn.param ncnnbin=runs/detect/train/weights/best_ncnn_model/model.ncnn.bin ncnnpy=runs/detect/train/weights/best_ncnn_model/model_ncnn.py pnnxparam=runs/detect/train/weights/best_ncnn_model/model.pnnx.param pnnxbin=runs/detect/train/weights/best_ncnn_model/model.pnnx.bin pnnxpy=runs/detect/train/weights/best_ncnn_model/model_pnnx.py pnnxonnx=runs/detect/train/weights/best_ncnn_model/model.pnnx.onnx fp16=0 device=cpu inputshape="[1, 3, 640, 640]"'



pnnxparam = runs/detect/train/weights/best_ncnn_model/model.pnnx.param
pnnxbin = runs/detect/train/weights/best_ncnn_model/model.pnnx.bin
pnnxpy = runs/detect/train/weights/best_ncnn_model/model_pnnx.py
pnnxonnx = runs/detect/train/weights/best_ncnn_model/model.pnnx.onnx
ncnnparam = runs/detect/train/weights/best_ncnn_model/model.ncnn.param
ncnnbin = runs/detect/train/weights/best_ncnn_model/model.ncnn.bin
ncnnpy = runs/detect/train/weights/best_ncnn_model/model_ncnn.py
fp16 = 0
optlevel = 2
device = cpu
inputshape = [1,3,640,640]f32
inputshape2 = 
customop = 
moduleop = 
get inputshape from traced inputs
inputshape = [1,3,640,640]f32
############# pass_level0
inline module = torch.nn.modules.linear.Identity
inline module = ultralytics.nn.modules.block.Attention
inline module = ultralytics.nn.modules.block.Bottleneck
inline module = ultralytics.nn.modules.block.C2PSA
inline module = ultralytics.nn.modules.block.C3k
inline module = ultralytics.nn.modules.block.C3k2
inline module = ultr

NCNN: export success ✅ 3.3s, saved as 'runs/detect/train/weights/best_ncnn_model' (10.0 MB)

Export complete (6.1s)
Results saved to /kaggle/working/runs/detect/train/weights
Predict:         yolo predict task=detect model=runs/detect/train/weights/best_ncnn_model imgsz=640  
Validate:        yolo val task=detect model=runs/detect/train/weights/best_ncnn_model imgsz=640 data=/kaggle/input/bird-train-valid-test-split/data.yaml  
Visualize:       https://netron.app
Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.1±0.1 ms, read: 74.6±33.5 MB/s, size: 36.1 KB)


...

val: Scanning /kaggle/input/bird-train-valid-test-split/test/labels... 961 images, 135 backgrounds, 0 corrupt: 100%|██████████| 961/961 [00:03<00:00, 258.71it/s]


WARNING ⚠️ val: Cache directory /kaggle/input/bird-train-valid-test-split/test is not writeable, cache not saved.


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 61/61 [00:06<00:00,  8.94it/s]
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        961       2921      0.809      0.321      0.578      0.284
Speed: 0.5ms preprocess, 4.0ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/detect/train3


array([    0.28397])